In [1]:
import pandas as pd
import numpy as np
import re


### Read in csv file for population and population casting and corresponding postcode and SA2

In [2]:
df_pop = pd.read_csv('../data/curated_data/vic_pop.csv')
df_pop_fore = pd.read_csv('../data/curated_data/population_forecast.csv')
df_post_SA2 = pd.read_csv('../data/raw_data/australian_postcodes.csv')


In [3]:
df_post_SA2 = df_post_SA2[df_post_SA2['state'] == 'VIC']
df_post_SA2 = df_post_SA2.loc[:,['postcode','SA2_MAINCODE_2016']]



### Base on our finding for property postcode select certain range to used

In [4]:
df_post_SA2 = df_post_SA2[df_post_SA2['postcode'] <= 4000]
df_post_SA2 = df_post_SA2.drop_duplicates()
df_post_SA2_dup = df_post_SA2[~df_post_SA2.duplicated('SA2_MAINCODE_2016')]


In [5]:
SA2 = 206041122.0
df_post_SA2[df_post_SA2['SA2_MAINCODE_2016']==SA2]

,postcode,SA2_MAINCODE_2016
6151,3000,206041122.0
6152,3001,206041122.0


In [6]:
SA2_list = df_post_SA2_dup['SA2_MAINCODE_2016'].values.tolist()
df_pop = df_pop.loc[:,['SA2','2021']]
df_new_pop = pd.merge(df_pop,df_pop_fore,how='left')

In [7]:
df_income = pd.read_csv('../data/curated_data/income.csv')
del_list = []
for i in SA2_list:
    if(df_new_pop[df_new_pop['SA2']==i].empty):
        del_list.append(i)
    elif(df_new_pop[df_new_pop['SA2']==i]['2021'].iat[0] == 0):
        del_list.append(i)


In [8]:
i = 206041122

df_income[df_income['SA2']==i]

,SA2,SA2 NAME,Earners,Median age of earners (years),Sum ($),Median ($),Mean ($)


In [9]:
df_new_pop[df_new_pop['SA2']==205021083]['2021']==0

82    True
Name: 2021, dtype: bool

In [10]:
df_income[df_income['SA2']==205021083]

,SA2,SA2 NAME,Earners,Median age of earners (years),Sum ($),Median ($),Mean ($)


In [11]:
for i in del_list:
    df_post_SA2 = df_post_SA2[df_post_SA2['SA2_MAINCODE_2016']!= i]

In [12]:
df_income = pd.read_csv('../data/curated_data/income.csv')


In [13]:
post_SA2 = df_post_SA2['population'] = 0
post_SA2 = df_post_SA2['fore_population'] = 0
post_SA2 = df_post_SA2['mean'] = 0
post_SA2 = df_post_SA2['median'] = 0
post_SA2 = df_post_SA2.loc[:,['postcode','population','fore_population','mean','median']]
all_post_code = post_SA2.drop_duplicates()

In [14]:
df_pro = pd.read_csv('../data/raw_data/new_properties.csv')
def get_postalcode(name):
    return int(name[-4:])
df_pro['postcode'] = df_pro.name.apply(get_postalcode) 

In [15]:

from re import L


def compute_ratio(postlist,property_df,i):
    total = len(property_df.loc[property_df['postcode'].isin(postlist)])
    num_i = len(property_df.loc[property_df['postcode'].isin([i])])
    if(total == 0):
        return 0
    ratio = num_i/total
    
    #@print(i, ratio)
    return ratio


def compute(SA2,all_post_code,property_df,population_df,income_df,del_list):
    if SA2 in del_list:
        return 0
    #print(SA2)
    # Population number in SA2 form
    pop2021 =  population_df[population_df['SA2'] == SA2].iloc[:,[1,-1]]['2021'].iat[0]
    temp_pop2026 =  population_df[ population_df['SA2'] == SA2].iloc[:,[1,-1]]['2026'].iat[0]
    pop2026 = int(re.sub(',',"",temp_pop2026))
    # total income number in SA2 form
    temp_median = df_income[df_income['SA2'] == str(SA2)].iloc[:,[-2,-1]]['Median ($)'].iat[0]
    temp_mean = df_income[df_income['SA2'] == str(SA2)].iloc[:,[-2,-1]]['Mean ($)'].iat[0]
    
    median = int(float(re.sub(',',"",temp_median)))
    mean = int(float(re.sub(',',"",temp_mean)))

    # step 1 find which postcodes own the input SA2
    post_list = df_post_SA2[df_post_SA2['SA2_MAINCODE_2016'] == SA2]['postcode'].values.tolist()
    #print(pop2021,pop2026,median,mean)
    
    # step 2 get corresponding ratio value for each post code

    for i in post_list:
        ratio = compute_ratio(post_list,property_df, i)
        all_post_code.loc[all_post_code['postcode']==i,'population']= all_post_code.loc[all_post_code['postcode']==i,'population'] + ratio*pop2021
        all_post_code.loc[all_post_code['postcode']==i,'fore_population']= all_post_code.loc[all_post_code['postcode']==i,'fore_population'] + ratio*pop2026
        all_post_code.loc[all_post_code['postcode']==i,'median']= all_post_code.loc[all_post_code['postcode']==i,'median'] + ratio*median
        all_post_code.loc[all_post_code['postcode']==i,'mean']= all_post_code.loc[all_post_code['postcode']==i,'mean'] + ratio*mean
    return 1
    


In [16]:
df_pro = pd.read_csv('../data/raw_data/new_properties.csv')
def get_postalcode(name):
    return int(name[-4:])
df_pro['postcode'] = df_pro.name.apply(get_postalcode) 
post_SA2 = df_post_SA2['population'] = 0
post_SA2 = df_post_SA2['fore_population'] = 0
post_SA2 = df_post_SA2['income'] = 0
post_SA2 = df_post_SA2.loc[:,['postcode','population','fore_population','median','mean']]
all_post_code = post_SA2.drop_duplicates()
SA2_list_new = []
for elem in SA2_list:
    if not np.isnan(elem):
        SA2_list_new.append(elem)
for i in SA2_list_new:
    break_num = compute(int(i),all_post_code,df_pro,df_new_pop,df_income,del_list)
    if(break_num == 0):
        continue

/tmp/ipykernel_12167/2987300627.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_post_code.loc[all_post_code['postcode']==i,'population']= all_post_code.loc[all_post_code['postcode']==i,'population'] + ratio*pop2021
/tmp/ipykernel_12167/2987300627.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_post_code.loc[all_post_code['postcode']==i,'fore_population']= all_post_code.loc[all_post_code['postcode']==i,'fore_population'] + ratio*pop2026
/tmp/ipykernel_12167/2987300627.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation:

In [17]:
all_post_code

,postcode,population,fore_population,median,mean
6151,3000,43823.000000,73594.000000,30024.000000,52476.000000
6152,3001,0.000000,0.000000,0.000000,0.000000
6153,3002,4962.000000,6519.000000,75755.000000,132130.000000
6155,3004,8797.876564,14591.749939,35776.034601,60208.315828
6158,3005,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...
9644,3990,0.000000,0.000000,0.000000,0.000000
9645,3991,0.000000,0.000000,0.000000,0.000000
9646,3992,1137.148936,1178.382979,1697.191489,2099.574468
9650,3995,12508.638298,12962.212766,18669.106383,23095.319149


In [18]:
df_final = pd.merge(df_pro,all_post_code,how='left')

https://www.abs.gov.au/statistics/labour/earnings-and-working-conditions/personal-income-australia/2014-15-2018-19#data-download

新的income 链接

In [19]:
df_final

,name,cost_text,coordinates,type,bond,features,unverified_features,desc,costs,beds,...,school,park,supermarket,cinema,postalcode,postcode,population,fore_population,median,mean
0,8092Y/33 Rose Lane Melbourne VIC 3000,$60 per week,"[-37.8150137, 144.9538694]",Apartment / Unit / Flat,261.0,[],[],['. Close to Southern Cross Station. Available...,60.0,1,...,11,170,48,7,VIC 3000,3000,43823.000000,73594.000000,30024.000000,52476.000000
1,602/118 Franklin Street Melbourne VIC 3000,$330,"[-37.8082052, 144.9589035]",Apartment / Unit / Flat,1320.0,[],[],['The exclusive address simply speaks for itse...,330.0,1,...,16,166,48,7,VIC 3000,3000,43823.000000,73594.000000,30024.000000,52476.000000
2,1112/333-351 Exhibition Street Melbourne VIC 3000,$600 per week,"[-37.8078956, 144.9682873]",Apartment / Unit / Flat,2607.0,['Furnished'],[],['Positioned in this most favourable location ...,600.0,2,...,19,173,54,6,VIC 3000,3000,43823.000000,73594.000000,30024.000000,52476.000000
3,3002/288 Spencer St Melbourne VIC 3000,$510,"[-37.813775, 144.9520948]",Apartment / Unit / Flat,2216.0,[],[],['This fantastic apartment located on the 30th...,510.0,2,...,10,108,44,6,VIC 3000,3000,43823.000000,73594.000000,30024.000000,52476.000000
4,4/180 Little Collins Street Melbourne VIC 3000,$500 per week,"[-37.8138601, 144.9679067]",Apartment / Unit / Flat,2173.0,['Furnished'],[],['Enjoy what the renowned reputation The Georg...,500.0,1,...,13,146,49,7,VIC 3000,3000,43823.000000,73594.000000,30024.000000,52476.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15231,16B Sandy Mount Avenue Inverloch VIC 3996,$400.00 per week,"[-38.6340011, 145.725239]",House,1738.0,[],"['Built in wardrobes', 'Internal Laundry', 'Se...","[""This spacious, two bedroom townhouse is perf...",400.0,2,...,0,0,0,0,VIC 3996,3996,8786.431082,8949.679926,22227.144311,27874.606846
15232,28 Beachcomber Drive Inverloch VIC 3996,$550 wk,"[-38.6412502, 145.7031282]",House,2390.0,[],"['Gas', 'Internal Laundry', 'Secure Parking', ...","[""The popular Beachcomber estate offers you mo...",550.0,4,...,0,0,0,0,VIC 3996,3996,8786.431082,8949.679926,22227.144311,27874.606846
15233,14 Inverloch Parade Inverloch VIC 3996,$420 wk,"[-38.6287143, 145.7358082]",House,1820.0,"['Garden', 'Split System AC', 'Floorboards']","['Internal Laundry', 'Heating']","['This two bedroom, 1 bathroom home will surel...",420.0,2,...,0,0,0,0,VIC 3996,3996,8786.431082,8949.679926,22227.144311,27874.606846
15234,Inverloch VIC 3996,$475 wk,"[-38.6314613, 145.7293638]",House,2065.0,"['Ensuite', 'Garden', 'Split System AC', 'Outd...",['Heating'],"[""Neat 3 bedroom home located in the immaculat...",475.0,3,...,0,0,0,0,VIC 3996,3996,8786.431082,8949.679926,22227.144311,27874.606846


In [20]:
df_final.to_csv('../data/curated_data/new_properties.csv',index = False)